In [2]:
import pandas as pd
from pmdarima import auto_arima
import matplotlib.pyplot as plt

# 특정 교량에서의 한 단위시간 뒤 하천수위 예측

In [8]:
# 시계열 데이터 불러오기
bridge = input("예측하고자 하는 교량 이름을 입력하시오 : ")
# df = pd.read_csv(f"실시간 {bridge}.csv")
df = pd.read_csv("C:\\Users\\lg\\Desktop\\고척교.csv", encoding='cp949')

# 피처 선택
if bridge == '고척교':
    df.drop('습도(%)', axis=1, inplace=True)
elif bridge == '도림교':
    df.drop('습도(%)', axis=1, inplace=True)
elif bridge == '성동교':
    df.drop(['풍속(m/s)', '강수량(mm)','습도(%)'], axis=1, inplace=True)
elif bridge == '기아대교':
    df.drop('습도(%)', axis=1, inplace=True)
elif bridge == '봉은교':
    df.drop(['풍속(m/s)','습도(%)'], axis=1, inplace=True)


# 시간 열을 datetime 형식으로 변환
df['시간'] = pd.to_datetime(df['시간'])

# 시간 열을 인덱스로 설정
df.set_index('시간', inplace=True)

# 하천 수위 데이터 선택
river_level_data = df['수위']

# auto_arima 함수로 최적 ARIMA 모델 차수 선택
model = auto_arima(river_level_data, seasonal=False, suppress_warnings=True)

# 최적 ARIMA 모델 예측
forecast_steps = 1
forecast, conf_int = model.predict(n_periods=forecast_steps, return_conf_int=True)

# 각 교량의 제방고 높이 정보
banggo = {
    "월계1교":21.16,
    "성동교":19.49,
    "마장2교":18.34,
    "고척교":15.44,
    "도림교":14.36,
    "기아대교":18.35,
    "봉은교":21.43
}

# 한 단위시간 뒤의 하천 수위 예측 결과 출력
print("예측된 하천 수위:", forecast[0])

# 범람 위험 시 문자 출력
if banggo[f"{bridge}"] < forecast[0]:
    print(f"전방 {bridge}는 범람 위험이 있으므로 주의하시길 바랍니다!")


예측하고자 하는 교량 이름을 입력하시오 : 고척교
예측된 하천 수위: 15.983999999956184
전방 고척교는 범람 위험이 있으므로 주의하시길 바랍니다!


# 예측 결과 시각화

In [ ]:
plt.plot(river_level_data, label='Original Data')
plt.plot(pd.date_range(start=river_level_data.index[-1], periods=forecast_steps+1, closed='right'), forecast, label='Forecast', color='red')
plt.fill_between(pd.date_range(start=river_level_data.index[-1], periods=forecast_steps+1, closed='right'), conf_int[:, 0], conf_int[:, 1], color='pink', alpha=0.3)
plt.legend()
plt.show()